In [9]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!sudo apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [9,558 B]
Get:12 http://ppa.launchpad.net/marutter

In [10]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-16 20:38:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.77MB/s    in 0.2s    

2020-08-16 20:38:15 (4.77 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DiabeticETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [12]:
from pyspark import SparkFiles
# Load in RuralData.csv from S3 into a DataFrame
url = "https://diabetic-by-county.s3.amazonaws.com/RuralData.csv"
spark.sparkContext.addFile(url)

rural_data = spark.read.option('header', 'true').csv(SparkFiles.get("RuralData.csv"), inferSchema=True, sep=',', header=True)
rural_data.show(10)

+----+-------+-------+---------+---------+------------+----------+----------+----------+----------+---------+------------+-------------+---------+------------------+------------------+-------------------+-------------------+--------+-------------+---------+-------------------+----------+---------------+-------+----------+
|FIPS|POP2010|OHU2010|TractLOWI|TractKids|TractSeniors|TractWhite|TractBlack|TractAsian|TractNHOPI|TractAIAN|TractOMultir|TractHispanic|TractHUNV|       PovertyRate|MedianFamilyIncome|           LA1and10|        LAhalfand10|LA1and20|LATracts_half|LATracts1|         LATracts10|LATracts20|         County|  State|Percentage|
+----+-------+-------+---------+---------+------------+----------+----------+----------+----------+---------+------------+-------------+---------+------------------+------------------+-------------------+-------------------+--------+-------------+---------+-------------------+----------+---------------+-------+----------+
|1001|25405.0| 9226.0|   922

In [13]:
# Load in UrbanData.csv from S3 into a DataFrame
url = "https://diabetic-by-county.s3.amazonaws.com/UrbanData.csv"
spark.sparkContext.addFile(url)

urban_data = spark.read.option('header', 'true').csv(SparkFiles.get("UrbanData.csv"), inferSchema=True, sep=',', header=True)
urban_data.show(10)

+----+-------+-------+---------+---------+------------+----------+----------+----------+----------+---------+------------+-------------+---------+------------------+------------------+------------------+-----------+------------------+-------------+------------------+----------+----------+---------------+-------+----------+
|FIPS|POP2010|OHU2010|TractLOWI|TractKids|TractSeniors|TractWhite|TractBlack|TractAsian|TractNHOPI|TractAIAN|TractOMultir|TractHispanic|TractHUNV|       PovertyRate|MedianFamilyIncome|          LA1and10|LAhalfand10|          LA1and20|LATracts_half|         LATracts1|LATracts10|LATracts20|         County|  State|Percentage|
+----+-------+-------+---------+---------+------------+----------+----------+----------+----------+---------+------------+-------------+---------+------------------+------------------+------------------+-----------+------------------+-------------+------------------+----------+----------+---------------+-------+----------+
|1001|29166.0|10995.0|   

# Postgres Setup

In [16]:
mode = 'append'
jdbc_url = "jdbc:postgresql://project3data.c9txthuk60be.us-east-1.rds.amazonaws.com:5432/urban_rural_data"
config = {"user": "root",
          "password": "bootcamp",
          "driver":"org.postgresql.Driver"}

In [17]:
rural_data.write.jdbc(url=jdbc_url, table='Rural', mode=mode, properties=config)

In [18]:
urban_data.write.jdbc(url=jdbc_url, table='Urban', mode=mode, properties=config)